In [ ]:
!pip install tabgan
!pip install -U scikit-learn
!pip install tabgan==1.1.0
# !pip install tabgan==1.0.3

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 82 kB 378 kB/s 
     |████████████████████████████████| 44 kB 2.4 MB/s 
  Attempting uninstall: tabgan
    Found existing installation: tabgan 1.1.1
    Uninstalling tabgan-1.1.1:
      Successfully uninstalled tabgan-1.1.1


In [ ]:
from tabgan.sampler import GANGenerator, OriginalGenerator 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import pandas as pd
import numpy as np

# random input data
train = pd.DataFrame(np.random.randint(-10, 150, size=(5000, 4)), columns=list("ABCD"))
target = pd.DataFrame(np.random.randint(0, 2, size=(5000, 1)), columns=list("Y"))
test = pd.DataFrame(np.random.randint(0, 100, size=(1000, 4)), columns=list("ABCD"))

# generate data
# new_train1, new_target1 = OriginalGenerator().generate_data_pipe(target, train, target, )
new_train2, new_target2 = GANGenerator(cat_cols=['A', 'B', 'C', 'D', 'Y'], 
                                       is_post_process=True, epochs=50
                                      #  adversarial_model_params={
                                      #          "metrics": "AUC", 
                                      #          "max_depth": 2,
                                      #          "max_bin": 100, 
                                      #          "n_estimators": 500,
                                      #          "learning_rate": 0.02, 
                                      #          "random_state": 42,
                                      #      }, 
                                      #  pregeneration_frac=2,
                                      #  epochs=50
                                       ).generate_data_pipe(train_df=train, 
                                                            test_df=test, 
                                                            target=target, 
                                                            only_adversarial=False,
                                                            use_adversarial=True,
                                                            )

Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/50 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
train

In [ ]:
new_target2

0       1
1       0
2       0
3       1
4       1
       ..
2495    0
2496    1
2497    1
2498    0
2499    1
Name: Y, Length: 2500, dtype: int64

In [ ]:
new_train2

,A,B,C,D
0,84,37,58,95
1,84,86,52,47
2,8,58,11,26
3,52,0,4,47
4,90,59,63,26
...,...,...,...,...
2495,35,68,89,10
2496,77,52,15,91
2497,79,79,39,7
2498,65,16,55,69


In [ ]:
target

In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np

if __name__ == "__main__":
    # random input data
    train = pd.DataFrame(np.random.randint(-10, 150, size=(50, 4)), columns=list('ABCD'))
    target = pd.DataFrame(np.random.randint(0, 2, size=(50, 1)), columns=list('Y'))
    test = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))

    # generate data
    # new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test, )
    # new_train1, new_target1 = GANGenerator().generate_data_pipe(train, target, test, )
    
    # example with all params defined
    new_train3, new_target3 = GANGenerator(gen_x_times=1.1, 
                                           cat_cols=None, 
                                           bot_filter_quantile=0.001,
                                           top_filter_quantile=0.999,
                                           is_post_process=True,
                                           adversarial_model_params={
                                               "metrics": "AUC", 
                                               "max_depth": 2,
                                               "max_bin": 100, 
                                               "n_estimators": 500,
                                               "learning_rate": 0.02, 
                                               "random_state": 42,
                                           }, 
                                           pregeneration_frac=2,
                                          #  epochs=500
                                           ).generate_data_pipe(train, 
                                                                target,
                                                                test, 
                                                                deep_copy=True,
                                                                only_adversarial=False,
                                                                use_adversarial=True)


In [ ]:
new_train1

In [ ]:
new_target1

In [ ]:
import sklearn
from sklearn import datasets
from sklearn import ensemble

In [ ]:
import sklearn
from sklearn import datasets
from sklearn import ensemble
def fit_predict(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    return sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])



dataset = sklearn.datasets.load_breast_cancer()
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=25, max_depth=6)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    pd.DataFrame(dataset.data), pd.DataFrame(dataset.target, columns=["target"]), test_size=0.33, random_state=42)
print("initial metric", fit_predict(clf, X_train, y_train, X_test, y_test))

new_train1, new_target1 = OriginalGenerator().generate_data_pipe(X_train, y_train, X_test, )
print("OriginalGenerator metric", fit_predict(clf, new_train1, new_target1, X_test, y_test))

new_train1, new_target1 = GANGenerator().generate_data_pipe(X_train, y_train, X_test, )
print("GANGenerator metric", fit_predict(clf, new_train1, new_target1, X_test, y_test))

In [ ]:
import torch

# print(torch.cuda.is_available())
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tabgan.sampler import OriginalGenerator, GANGenerator

# read workers compensation data
X = pd.read_excel("https://datarobotfiles.s3-ap-southeast-1.amazonaws.com/20210502X_train3.xlsx")
y = pd.read_csv("https://datarobotfiles.s3-ap-southeast-1.amazonaws.com/20210502y_target.csv", low_memory=False)

# one hot encoding
X = pd.get_dummies(X)

# training, test, and target split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Finished setting up data!")

# generate data
new_train2, new_target2 = GANGenerator().generate_data_pipe(X_train, y_train, X_test, )
print("Finished generating data!")

In [ ]:
from tabgan import sampler

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    pd.DataFrame(dataset.data), pd.DataFrame(dataset.target, columns=["target"]), test_size=0.33, random_state=42)
new_train2, new_target2 = sampler.GANGenerator().generate_data_pipe(x_train, y_train, x_test, only_adversarial=False)
new_train3, new_target3 = sampler.OriginalGenerator().generate_data_pipe(x_train, y_train, x_test)

In [ ]:
new_train2

In [ ]:
new_train3